<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENT_claude_3_7_sonnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://docs.anthropic.com/claude/reference/client-sdks

* Claude 3 Opus: This is their most intelligent model, designed for highly complex tasks, open-ended prompts, and scenarios requiring human-like understanding.

* Claude 3 Sonnet (and Claude 3.7 Sonnet): This model offers a strong balance of performance and speed, suitable for enterprise workloads and high-throughput tasks. Claude 3.7 Sonnet is their latest and most intelligent model, featuring hybrid reasoning and an adjustable "thinking budget" for cost management.

* Claude 3 Haiku: This is their fastest and most cost-effective model, designed for near-instant responsiveness in tasks like live customer chats and quick information retrieval

In [ ]:
!pip install anthropic -q

!pip install colab-env --quiet

In [ ]:
import anthropic
import os
import colab_env
import json

https://docs.anthropic.com/en/docs/models-overview

In [3]:
def claude_chat(prompt,model,client):
    message = client.messages.create(
    model="claude-3-7-sonnet-20250219", # 30/04/2005
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    print()
    print("-" * 80)
    print('Question: %s'%prompt)
    print("-" * 80)
    print()
    print('Answer: ')
    print(message.content[0].text)
    #return message.content[0].text

In [4]:
api_key = os.environ["CLAUDE3_API_KEY"]
model="claude-3-7-sonnet-20250219", # 30/04/2005

client = anthropic.Anthropic(
    api_key=api_key,
)

In [5]:
message = client.messages.create(
    model="claude-3-7-sonnet-20250219", # 30/04/2005
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)

In [8]:
print(message.content[0].text)

Hello! How can I assist you today? I'm ready to help with information, answer questions, or discuss any topics you'd like to explore. What's on your mind?


In [6]:
import asyncio
from typing import Optional, List, Dict, Any
from anthropic import Anthropic
import os
import datetime

class ClaudeFlightAgent:
    """
    An AI agent exclusively for flight planning using the Anthropic Claude API.
    This version is strictly limited to flight-related information retrieval and filtering.
    """

    def __init__(self, api_key: str):
        """
        Initializes the ClaudeFlightAgent.

        Args:
            api_key (str): The Anthropic Claude API key.
        """
        self.api_key = api_key
        self.model="claude-3-7-sonnet-20250219" # 30/04/2005

        self.anthropic = Anthropic(api_key=self.api_key)
        self.available_tools = [{"name": "find_flights",
                                 "description": "Finds flight options based on user criteria.",
                                 "input_schema": {
                                     "type": "object",
                                     "properties": {
                                         "origin": {"type": "string", "description": "Origin airport code (e.g., YVR)"},
                                         "destination": {"type": "string", "description": "Destination airport code (e.g., SAT)"},
                                         "departure_date": {"type": "string", "format": "date",
                                                            "description": "Departure date (YYYY-MM-DD)"},
                                         "return_date": {"type": "string", "format": "date",
                                                         "description": "Return date for round trip (YYYY-MM-DD)"},
                                         "budget": {"type": "number", "description": "Maximum budget for the flights"},
                                         "num_passengers": {"type": "integer",
                                                            "description": "Number of passengers"},
                                         "preferred_airlines": {"type": "array",
                                                              "items": {"type": "string"},
                                                              "description": "List of preferred airlines"},
                                         "cabin_class": {"type": "string",
                                                         "description": "Cabin class (e.g., economy, business)"},
                                         "sort_by": {"type": "string",
                                                     "enum": ["price", "departure_time", "arrival_time",
                                                              "duration", "stops"],
                                                     "description": "Sort results by: price, departure_time, arrival_time, duration, or stops"},
                                         "max_stops": {"type": "integer",
                                                       "description": "Maximum number of stops"},
                                         "currency": {"type": "string",
                                                      "description": "Currency for prices (e.g., USD, EUR, CAD)"}
                                     },
                                     "required": ["origin", "destination"]
                                 }}]  # Strictly flight-focused tool schema

    async def claude_chat(self, messages: List[dict], tools: Optional[List[dict]] = None, max_tokens: int = 1000) -> dict:
        """Sends messages to the Claude API and returns the full response."""
        #print("Messages being sent to the model:")  # Print the messages before sending
        #print(json.dumps(messages, indent=4)) # Print the structure of messages
        response = self.anthropic.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            messages=messages,
            tools=tools
        )
        #print("Model response:")  # Print the entire response
        #print(json.dumps(response.model_dump(), indent=4))
        return response.model_dump()

    async def execute_tool_call(self, tool_call: dict) -> dict:
        """
        Executes a tool call and returns the result.

        Args:
            tool_call (dict): The tool call information.

        Returns:
            dict: The result of the tool call.
        """
        tool_name = tool_call["name"]
        tool_args = tool_call["input"]

        if tool_name == "find_flights":
            return self.find_flights_tool(tool_args)
        else:
            raise ValueError(f"Unknown tool: {tool_name}")

    def find_flights_tool(self, tool_input: dict) -> dict:
        """
        Simulates finding flight options.  Replaces external API call.

        Args:
            tool_input (dict): The input arguments for the flight search.

        Returns:
            dict: A dictionary containing flight options.
        """
        origin = tool_input.get("origin")
        destination = tool_input.get("destination")
        departure_date = tool_input.get("departure_date")
        return_date = tool_input.get("return_date")
        budget = tool_input.get("budget")
        num_passengers = tool_input.get("num_passengers", 1)
        preferred_airlines = tool_input.get("preferred_airlines", [])
        cabin_class = tool_input.get("cabin_class")
        sort_by = tool_input.get("sort_by")
        max_stops = tool_input.get("max_stops")
        currency = tool_input.get("currency")

        #  Simplified flight data (replace with actual flight API integration)
        flight_options = [
            {"airline": "AirLine 1", "flight_number": "123", "departure_time": "08:00",
             "arrival_time": "10:00", "price": 200, "stops": 0, "duration": 2, "cabin_class": "economy"},
            {"airline": "AirLine 2", "flight_number": "456", "departure_time": "10:00",
             "arrival_time": "12:00", "price": 250, "stops": 1, "duration": 2, "cabin_class": "business"},
            {"airline": "AirLine 3", "flight_number": "789", "departure_time": "09:00",
             "arrival_time": "11:30", "price": 180, "stops": 0, "duration": 2.5, "cabin_class": "economy"},
            {"airline": "AirLine 4", "flight_number": "101", "departure_time": "14:00",
             "arrival_time": "16:30", "price": 300, "stops": 2, "duration": 2.5, "cabin_class": "first"},
            {"airline": "AirLine 5", "flight_number": "222", "departure_time": "11:00",
             "arrival_time": "13:00", "price": 220, "stops": 0, "duration": 2, "cabin_class": "economy"}
        ]

        #  Filter flights
        filtered_flights = []
        for flight in flight_options:
            if budget is not None and flight["price"] > budget:
                continue
            if max_stops is not None and flight["stops"] > max_stops:
                continue
            if preferred_airlines and flight["airline"] not in preferred_airlines:
                continue
            if cabin_class and flight["cabin_class"] != cabin_class:
                continue
            filtered_flights.append(flight)

        # Sort flights
        if sort_by:
            filtered_flights.sort(key=lambda x: x.get(sort_by, 0))  # Sort safely

        # Format prices with currency
        if currency:
            for flight in filtered_flights:
                flight["price"] = f"{currency} {flight['price']}"

        return {"content": f"Found {len(filtered_flights)} flight options.", "flights": filtered_flights}

    # ... (other imports and class definition)

    async def process_query(self, query: str) -> str:
        """Processes a query using Claude and available tools."""
        messages = [{"role": "user", "content": query}]
        tools = self.available_tools

        while True:
            claude_response = await self.claude_chat(messages, tools=tools)
            final_text = []

            for response_item in claude_response["content"]:
                if response_item["type"] == "text":
                    final_text.append(response_item["text"])
                elif response_item["type"] == "tool_use":
                    tool_name = response_item.get("name")
                    tool_args = response_item.get("input")

                    if tool_name:
                        tool_result = await self.execute_tool_call({"name": tool_name, "input": tool_args})
                        final_text.append(f"\n[Tool Call: {tool_name} with args {tool_args}]")
                        final_text.append(f"\n[Tool Result: {tool_result['content']}]")

                        # Extract and display flight details (if available)
                        if tool_result.get("flights"):
                            final_text.append("\nFlight Options:")
                            for flight in tool_result["flights"]:
                                final_text.append(f"- {flight}")  # Format flight details as needed

                        # Send the tool result back to Claude with assistant role and tool_code
                        messages.append({
                            "role": "assistant", # Changed to assistant
                            "content": [
                                {
                                    "type": "text",
                                    "text": tool_result['content']
                                }
                            ]
                        })
                        break # break out of inner loop to send new message
            else:
                break # Stop if the model does not want to use a tool.

            if claude_response["stop_reason"] != "tool_use":
                break # Exit loop if Claude doesn't request another tool use

        return "\n".join(final_text)

    # ... (rest of your code)




async def main():
    """Main function to run the ClaudeFlightAgent."""

    api_key = os.environ["CLAUDE3_API_KEY"]

    planner = ClaudeFlightAgent(api_key)

    # Define the query here
    query = "Find the cheapest flights from JFK to LAX on 2024-12-20 in economy class for 2 people"
    #SAT from YVR

    #query = "Find the cheapest flights from YVR to SAT on 2024-12-20 in economy class for 2 people"

    response = await planner.process_query(query)
    print("\nAgent: " + response)  # Print the response


if __name__ == "__main__":
    #asyncio.run(main())
    import nest_asyncio
    nest_asyncio.apply() # allows nested event loop
    asyncio.get_event_loop().run_until_complete(main()) # using the existing event loop to run main


Agent: 

### Option 1: $346 total - JetBlue
- Departure: Thu Dec 20 - 8:15 PM (JFK)
- Arrival: Thu Dec 20 - 11:42 PM (LAX) 
- Duration: 6h 27m
- Nonstop
- Economy Class

### Option 2: $358 total - Spirit Airlines
- Departure: Thu Dec 20 - 6:05 AM (JFK)
- Arrival: Thu Dec 20 - 9:29 AM (LAX)
- Duration: 6h 24m
- Nonstop
- Economy Class

### Option 3: $396 total - Delta Airlines
- Departure: Thu Dec 20 - 1:00 PM (JFK)
- Arrival: Thu Dec 20 - 4:25 PM (LAX)
- Duration: 6h 25m
- Nonstop
- Economy Class

Would you like me to help you book any of these options, or would you like to see more flight choices with different criteria?
